Rui Wang 1007791695

# Homework 6: Part I

1. Go get data from kaggle.com and do a ***(Univariate) Bayesian Logistic Regression*** analysis

2. Adjust the code below to specify that the outcomes have a Bernoulli distribution and use a ***logit*** or ***probit link function*** (or $\Pr(z\leq 0)$ for latent $z$ ) to correctly paramterize the predicted values of the observed outcomes

```python
import pymc as pm; import numpy as np
n,p=100,10; X,y=np.zeros((n,p)),np.ones((n,1))
# Replace this made up data with your data set from kaggle...
with pm.Model() as MLR:
    betas = pm.MvNormal('betas', mu=np.zeros((p,1)), cov=np.eye(p), shape=(p,1))
    sigma = pm.TruncatedNormal('sigma', mu=1, sigma=1, lower=0) # half normal
    y = pm.Normal('y', mu=pm.math.dot(X, betas), sigma=sigma, observed=y)

with MLR:
    idata = pm.sample()
```    

3. Choose ***prior*** that are sensible for your specification

4. [Optional] Assess the performance of the MCMC and any issues or warnings in the [standard manner](https://www.pymc.io/projects/docs/en/stable/learn/core_notebooks/pymc_overview.html#pymc-overview)

4. [Optional] Go get data from kaggle.com and do a ***Multivariate Bayesian Logistic Regression*** analysis

# Part 0 #

I have found this dataset about Clothes Price Prediction in Kaggle, and here is the link to the dataset: https://www.kaggle.com/datasets/teertha/ushealthinsurancedataset

This dataset contains 1338 rows of insured data, where the Insurance charges are given against the following attributes of the insured: Age, Sex, BMI, Number of Children, Smoker and Region. The attributes are a mix of numeric and categorical variables.

PS: I used this data set from HW3 and HW5


In [2]:
import random
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import requests

df = pd.read_csv("insurance.csv")

df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


I choose age as major parameter.
(Chosen age as X, and bmi as y)

# Part 1

In [5]:
import pymc as pm; import numpy as np
n,p=1338,39; X,y=np.zeros((n,p)),np.ones((n,1))
with pm.Model() as logistic_model:
    betas = pm.MvNormal('betas', mu=np.zeros((p,1)), cov=np.eye(p), shape=(p,1))
    linear_comb = pm.math.dot(X, betas)
    p = pm.math.sigmoid(linear_comb)
    y_obs = pm.Bernoulli('y_obs', p=p, observed=y)
with logistic_model:
    idata = pm.sample()

# Homework 6: Part II<br>Regularized Loss Functions

***Machine Learning*** fits models by optimizing penalized ***loss functions***

Two classic regularizations are "ridge" and "lasso" regression, which respectively use $L_2$ and $L_1$ penalty functions

- Lasso: $$\sum_{i=1}^n \frac{1}{2}(y_i-x_i^T\beta_{p \times 1})^2 + \lambda \sum_{j=1}^n \beta_j^2 = \frac{1}{2}(y-X\beta)^T(y-X\beta) + \lambda \sum_{j=1}^n \beta_j^2 = \frac{1}{2}||y-X\beta||_2^2 + \lambda ||\beta||_2^2 $$
- Ridge: $$\sum_{i=1}^n \frac{1}{2}(y-x_i^T\beta_{p \times 1})^2 + \lambda \sum_{j=1}^n |\beta_j| = \frac{1}{2}(y-X\beta)^T(y-X\beta) + \lambda \sum_{j=1}^n |\beta_j| = \frac{1}{2}||y-X\beta||_2^2 + \lambda ||\beta||_1$$
    
Show that for $\sigma=1$ and ***hyperparameters*** $b_i=0$ (ignoring normalizing proportionality constants) the log posterior distributions for $\beta$ using either ***normal*** or ***Laplace*** prior distributions have analagous forms to the above expressions

Now write down and understand the following: "Bayesians do not optimize posterior distributions, they sample from them; but, the posterior distributions are nonetheless 'regularizations' of the likelihood through the prior."

# Part 2


In [6]:
import pymc as pm
import numpy as np
import arviz as az

# Set seed(Student ID)
np.random.seed(1007791695)
n_samples = 1000
X = np.random.normal(size=(n_samples, 1))
true_beta = 5
mu = X * true_beta
w = 1
nu = 5

y = np.random.normal(mu.flatten(), scale=w)

with pm.Model() as robust_model_with_lambda:
    beta = pm.Normal('beta', mu=0, sigma=10)
    nu = pm.Exponential('nu', 1/29) + 1
    lambda_i = pm.Gamma('lambda', alpha=nu/2, beta=nu/2, shape=n_samples)
    sigma_i = pm.Deterministic('sigma_i', 1 / (lambda_i))

    likelihood = pm.Normal('y', mu=X[:,0]*beta, sigma=sigma_i, observed=y)

    trace = pm.sample(1000, target_accept=0.9, return_inferencedata=True)

lambda_posterior = trace.posterior['lambda'].values
lambda_mean = np.mean(lambda_posterior, axis=(0, 1))
outlier_threshold = np.quantile(lambda_mean, 0.05)  # Lower 5% as potential outliers
outliers = np.where(lambda_mean < outlier_threshold)[0]
print(f"Potential outliers: {outliers}")

Potential outliers: [ 18  20  23  30  50  52  80  95 163 202 204 206 255 296 323 337 354 365
 409 411 435 458 464 470 504 511 516 519 540 547 553 578 596 599 676 712
 720 729 735 745 774 801 829 845 864 866 889 899 924 952]
